In [1]:
import os
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", message=".*defaulting to pandas implementation.*")
pd.set_option('display.float_format', '{:.6f}'.format)
start_time = time.time()

# Import Model

In [2]:
jupyter_dir = os.path.abspath(str(Path()))
jupyter_dir

'E:\\git\\machine-learning-option-pricing'

In [3]:
models_path = os.path.join(jupyter_dir,'trained_models','trained_models')
models = os.listdir(models_path)
for i, path in enumerate(models):
    print(f"{i}   {path}")

0   2024_10_09 23-01-12 Deep Neural Network otm
1   2024_10_09 23-11-23 Deep Neural Network


In [4]:
model = models[0]
model

'2024_10_09 23-01-12 Deep Neural Network otm'

In [5]:
model_path = os.path.join(models_path,model)

In [6]:
model_files = os.listdir(model_path)
model_files

['2024_10_09 23-01-12 Deep Neural Network otm insample.csv',
 '2024_10_09 23-01-12 Deep Neural Network otm outsample.csv',
 '2024_10_09 23-01-12 Deep Neural Network otm.pkl',
 '2024_10_09 23-01-12 Deep Neural Network otm.txt']

In [7]:
spec_path = os.path.join(model_path,[file for file in model_files if file.endswith('.txt')][0])

In [8]:
model_files

['2024_10_09 23-01-12 Deep Neural Network otm insample.csv',
 '2024_10_09 23-01-12 Deep Neural Network otm outsample.csv',
 '2024_10_09 23-01-12 Deep Neural Network otm.pkl',
 '2024_10_09 23-01-12 Deep Neural Network otm.txt']

In [9]:
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
outsample_path = os.path.join(model_path,[file for file in model_files if file.find('outsample') != -1][0])
outsample = pd.read_csv(outsample_path).iloc[:,1:]
outsample

KeyboardInterrupt: 

# Test Model

In [ ]:
maturities = outsample['days_to_maturity'].unique().tolist()
barrier_type_names = outsample['barrier_type_name'].unique().tolist()
ws = outsample['w'].unique().tolist()
print(f"\nmaturities:\n     maturities = {maturities}")
print(f"\ntypes:\n     barrier_type_names = {barrier_type_names}\n     ws = {ws}\n")

In [ ]:
by_wbt = outsample.copy().set_index(['w','barrier_type_name','days_to_maturity']).sort_index()

In [ ]:
RMSE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    columns = barrier_type_names,
    index = maturities
)

for t in maturities:
    for b in barrier_type_names:
        RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(slice(None), b, t),'outofsample_error'])**2))
print(f"\nRMSE:\n{RMSE}")

In [ ]:
MAE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    index = maturities,
    columns = barrier_type_names
)
for t in maturities:
    for b in barrier_type_names:
        MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(slice(None), b, t),'outofsample_error']))
print(f"\nMAE:\n{MAE}")

In [ ]:
plt.figure()
for b in barrier_type_names:
    plt.plot(MAE.loc[:, b], label=b)
plt.xlabel('maturities in days')
plt.ylabel('RMSE')
plt.legend()
plt.show()
plt.close()

In [ ]:
errors_bypayoff = {}
for w in ws:
    errors = {}
    print(
        f"\n         {'#'*(len(w)+5)}"
        f"\n         # {w}s #"
        f"\n         {'#'*(len(w)+5)}"
    )
    RMSE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
        )
    RMSE[w] = maturities
    RMSE = RMSE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(w, b, t),'outofsample_error'])**2))
    print(f"RMSE:\n{RMSE}")
    errors['RMSE'] = RMSE
    


    MAE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
    )
    MAE[w] = maturities
    MAE = MAE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(w, b, t),'outofsample_error']))
    print(f"MAE:\n{MAE}")
    errors['MAE'] = MAE

    errors_bypayoff[w] = errors
    print("\n\n")

In [ ]:
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        print(errors['MAE'])

In [ ]:
plt.figure()
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        MAE = errors['MAE'][b]
        plt.plot(
            MAE.sort_values(), label=f"{b} {w}s ",
        )
plt.legend(loc='best')
plt.show()
plt.close()

In [ ]:
end_time = time.time()
runtime = end_time - start_time
print(f"cpu: {runtime}")